## Interacions
Estraggo le interacions dal dataset dei commenti 

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

data=pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_comments.csv',encoding='ISO-8859-1')
post_count = data.groupby('post_id')['from_id'].nunique().reset_index(name='post_size')
post_from_count = data.groupby(['post_id', 'from_id']).size().reset_index(name='interaction_len')
joined_df = pd.merge(post_count, post_from_count, on='post_id', how='inner')
joined_df[['post_size','interaction_len']].to_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_interactions.csv')

## Outreach for each post

I successivi blocchi di codice consentono di fare un confronto tra la len dell'interazione e la size dell'outreach di una pagina misurata come Lurkers in quel momento.

In [98]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv',usecols=['created_time', 'page_id', 'post_id'],encoding='ISO-8859-1')
likes = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_likes.csv', usecols=['from_id', 'post_id'], encoding='ISO-8859-1')

In [2]:
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv',usecols=['created_time', 'page_id', 'post_id'],encoding='ISO-8859-1')
likes = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_likes.csv', usecols=['from_id', 'post_id'], encoding='ISO-8859-1')

Salvo per ciascuna pagine la serie storica dei Lurkers, ovvero il numero di utenti univoci che ha lasciato like a un qualsiasi post della pagina.

In [42]:
from tqdm import tqdm
import pandas as pd

# Assuming posts and likes DataFrames are already defined
for page_id in tqdm(posts['page_id'].unique()):
    page_posts = posts[posts['page_id'] == page_id]
    page_likes = likes[likes['post_id'].isin(page_posts['post_id'].unique())]
    complete_page_likes = pd.merge(page_posts, page_likes, on='post_id', how='right')
    complete_page_likes['created_time'] = pd.to_datetime(complete_page_likes['created_time'])
    complete_page_likes.set_index('created_time', inplace=True)
    weekly_unique_users = complete_page_likes['from_id'].resample('W').nunique()
    monthly_moving_avg = weekly_unique_users.rolling(window=4).mean()
    monthly_avg_df = pd.DataFrame({
        'Date': monthly_moving_avg.index,
        'Monthly Moving Average': monthly_moving_avg.values
    })
    monthly_avg_df = monthly_avg_df.dropna()
    monthly_avg_df.to_csv(f'/home/jacoponudo/Documents/Size_effects/DATA/page_size/{page_id}_monthly_moving_avg.csv', index=False)


100%|██████████| 117/117 [08:13<00:00,  4.22s/it]


In [211]:

folder_path = '/home/jacoponudo/Documents/Size_effects/DATA/page_size/'
dataframes = []
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith('_monthly_moving_avg.csv'):
        page_id = filename.split('_')[0]  # Estrae page_id dal nome del file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        df.insert(2, 'page_id', page_id)
        dataframes.append(df)
outreach = pd.concat(dataframes, ignore_index=True)
output_file = '/home/jacoponudo/Documents/Size_effects/DATA/page_size/all_outreach_pages.csv'
outreach.to_csv(output_file, index=False)


100%|██████████| 118/118 [00:00<00:00, 1367.47it/s]
/tmp/ipykernel_47432/501097947.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  outreach = pd.concat(dataframes, ignore_index=True)


Puliamo la ram da tutti i file in memoria.

In [44]:
import gc
def clear_all():
    for name in dir():
        if not name.startswith('_'):
            del globals()[name]
    gc.collect()

# Pulisci la RAM da tutte le variabili
clear_all()

Per ogni interazione, calcolo quanto dura (interaction len) e dove avviene (page_id) e quando è iniziata (created_time), e qual'era il valore di Luckers sulla pagina su cui avviene in quel momento.

In [51]:
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv',usecols=['created_time', 'page_id', 'post_id'],encoding='ISO-8859-1')
post_id_to_page_id = posts.set_index('post_id')['page_id'].to_dict()
del posts
gc.collect()

In [59]:
comments = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_comments.csv',encoding='ISO-8859-1',usecols=['from_id', 'created_time', 'post_id'])
interactions = comments.groupby(['from_id', 'post_id']).agg(count=('created_time', 'size'),first_created_time=('created_time', 'first')).reset_index()
del comments
gc.collect()
interactions['page_id'] = interactions['post_id'].map(post_id_to_page_id) # non runnata 

In [226]:
outreach['page_id'] = outreach['page_id'].astype(int)
interactions['page_id'] = interactions['page_id'].astype(int)


outreach_column=[]

for _, interaction in interactions.iterrows():
    outreach_page = outreach[outreach['page_id'] == interaction['page_id']]
    if len(outreach_page)<1:
        outreach_column.append(np.nan)
    

Empty DataFrame
Columns: [Date, Monthly Moving Average, page_id]
Index: []
                            Date  Monthly Moving Average      page_id
20542  2010-06-27 00:00:00+00:00                    3.00  35281584398
20543  2010-07-04 00:00:00+00:00                    3.25  35281584398
20544  2010-07-11 00:00:00+00:00                    3.50  35281584398
20545  2010-07-18 00:00:00+00:00                    5.00  35281584398
20546  2010-07-25 00:00:00+00:00                    5.50  35281584398
...                          ...                     ...          ...
20826  2015-12-06 00:00:00+00:00                 3816.75  35281584398
20827  2015-12-13 00:00:00+00:00                 3488.75  35281584398
20828  2015-12-20 00:00:00+00:00                 2946.00  35281584398
20829  2015-12-27 00:00:00+00:00                 3456.25  35281584398
20830  2016-01-03 00:00:00+00:00                 3542.50  35281584398

[289 rows x 3 columns]
Empty DataFrame
Columns: [Date, Monthly Moving Average, page_

KeyboardInterrupt: 

In [230]:
date_target = pd.to_datetime('2010-02-21 00:00:00+00:00')
closest_date = outreach_page['Date'].iloc[(outreach_page['Date'] - date_target).abs().argsort()[:1]]

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'

In [225]:
outreach_page


NameError: name 'outreach_page' is not defined

TypeError: string indices must be integers